In [1]:
from collections import deque
import os
import random
from tqdm import tqdm

import torch

from utils_drl import Agent
from utils_env import MyEnv
from utils_memory import ReplayMemory, PRMemory

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

use_dueling = False
use_PR = True
use_DDQN = False

In [2]:
GAMMA = 0.99
GLOBAL_SEED = 0
MEM_SIZE = 100_000
RENDER = False
SAVE_PREFIX = "./models"
STACK_SIZE = 4

EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 1000000

BATCH_SIZE = 32
POLICY_UPDATE = 4
TARGET_UPDATE = 10_000
WARM_STEPS = 50_000
MAX_STEPS = 50_000_000
EVALUATE_FREQ = 100_000

rand = random.Random()
rand.seed(GLOBAL_SEED)
new_seed = lambda: rand.randint(0, 1000_000)
if not os.path.exists(SAVE_PREFIX):
    os.mkdir(SAVE_PREFIX)

torch.manual_seed(new_seed())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = MyEnv(device)
agent = Agent(
    env.get_action_dim(),
    device,
    GAMMA,
    new_seed(),
    EPS_START,
    EPS_END,
    EPS_DECAY,
    use_dueling = use_dueling,
    use_PR = use_PR
)
if not use_PR:
    memory = ReplayMemory(STACK_SIZE + 1, MEM_SIZE, device)
else:
    memory = PRMemory(STACK_SIZE + 1, MEM_SIZE, device)



In [ ]:
#### Training ####
obs_queue: deque = deque(maxlen=5)
done = True

progressive = tqdm(range(MAX_STEPS), total=MAX_STEPS,
                   ncols=50, leave=False, unit="b")
for step in progressive:
    if done:
        observations, _, _ = env.reset()
        for obs in observations:
            obs_queue.append(obs)

    training = len(memory) > WARM_STEPS
    state = env.make_state(obs_queue).to(device).float()
    action = agent.run(state, training)
    obs, reward, done = env.step(action)
    obs_queue.append(obs)
    memory.push(env.make_folded_state(obs_queue), action, reward, done)

    if step % POLICY_UPDATE == 0 and training:
        if not use_PR:
            agent.learn(memory, BATCH_SIZE)
        else:
            agent.learn_PR(memory, BATCH_SIZE)

    if step % TARGET_UPDATE == 0:
        agent.sync()

    if step % EVALUATE_FREQ == 0:
        avg_reward, frames = env.evaluate(obs_queue, agent, render=RENDER)
        with open("rewards.txt", "a") as fp:
            fp.write(f"{step//EVALUATE_FREQ:3d} {step:8d} {avg_reward:.1f}\n")
        if RENDER:
            prefix = f"eval_{step//EVALUATE_FREQ:03d}"
            os.mkdir(prefix)
            for ind, frame in enumerate(frames):
                with open(os.path.join(prefix, f"{ind:06d}.png"), "wb") as fp:
                    frame.save(fp, format="png")
        agent.save(os.path.join(
            SAVE_PREFIX, f"model_{step//EVALUATE_FREQ:03d}"))
        done = True

  1%| | 409539/50000000 [1:55:10<266:13:12, 51.74b/s] 

In [4]:
batch_size=32
state_batch, action_batch, reward_batch, next_batch, done_batch, idxs, ISWeights = memory.sample(batch_size)

values = agent.__policy(state_batch.float()).gather(1, action_batch)
values_next = agent.__target(next_batch.float()).max(1).values.detach()
expected = (agent.__gamma * values_next.unsqueeze(1)) * (1. - done_batch) + reward_batch
            
abs_errors = torch.abs(expected - values).data.cpu().numpy()

AttributeError: 'Agent' object has no attribute '__policy'

In [5]:
ISWeights

tensor([[0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0067],
        [0.0011],
        [0.0120],
        [0.0046],
        [0.0053],
        [0.0088],
        [0.0058],
        [0.0086],
        [0.0025],
        [0.0036],
        [0.0086],
        [0.0042],
        [0.0048],
        [0.0221],
        [0.0037],
        [0.0034],
        [0.0042],
        [0.0024],
        [0.0032],
        [0.0269],
        [0.0027],
        [0.0090],
        [0.0013],
        [0.0067],
        [0.0022],
        [0.0051],
        [0.0019],
        [0.0010]], device='cuda:0')

In [21]:
memory.tree.get_min()

0.001007135258987546

In [19]:
1/MEM_SIZE/min_prob

187.83919137775848

In [15]:
import numpy as np
n=32
pri_seg = memory.tree.total_p / n       # priority segment
memory.beta = np.min([1., memory.beta + memory.beta_increment_per_sampling])  # max = 1

min_prob = memory.tree.get_min() / memory.tree.total_p     # for later calculate ISweight
if min_prob == 0:
    min_prob = 0.00001 / memory.tree.total_p
for i in range(n):
    a, b = pri_seg * i, pri_seg * (i + 1)
    v = np.random.uniform(a, b)
    idx, p, data = memory.tree.get_leaf(v)
    prob = p / memory.tree.total_p
    ISWeights[i, 0] = np.power(prob/min_prob, -memory.beta)
    #print(b_states[i].shape, data[0][0:4].shape)
    print(1/prob/MEM_SIZE , p ,ISWeights[i, 0])

1.3888310072684988 0.13621489703655243 tensor(0.0074, device='cuda:0')
0.18917947265625 1.0 tensor(0.0010, device='cuda:0')
0.18917947265625 1.0 tensor(0.0010, device='cuda:0')
0.18917947265625 1.0 tensor(0.0010, device='cuda:0')
0.694941048779361 0.2722237706184387 tensor(0.0037, device='cuda:0')
0.37210651845112763 0.5084013938903809 tensor(0.0020, device='cuda:0')
0.26256263146269637 0.720511794090271 tensor(0.0014, device='cuda:0')
1.749586569519624 0.10812810063362122 tensor(0.0093, device='cuda:0')
3.3081405345428454 0.05718604475259781 tensor(0.0176, device='cuda:0')
1.1463828804292961 0.16502293944358826 tensor(0.0061, device='cuda:0')
1.506545290816693 0.12557171285152435 tensor(0.0080, device='cuda:0')
0.7791874310698271 0.24279071390628815 tensor(0.0041, device='cuda:0')
0.6026656460260472 0.3139045238494873 tensor(0.0032, device='cuda:0')
0.3081877845731952 0.6138448119163513 tensor(0.0016, device='cuda:0')
0.6099925844511421 0.31013405323028564 tensor(0.0032, device='cuda:

In [13]:
max(memory.tree.tree[-MEM_SIZE:])

1.0